Importing needed modules:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as pyplot

import bc_utils as butils
from sklearn.datasets.samples_generator import make_blobs
from scipy.spatial import distance

Creating a test cluster:

In [ ]:
X, y = make_blobs(n_samples=300, centers=2, cluster_std=1.9, n_features=5)
df = pd.DataFrame(dict(a=X[:,0], b=X[:,1],c=X[:,2], d=X[:,3],e=X[:,4]))

Plotting test cluster for visualisation:

In [ ]:
df.head()

In [ ]:
#plt = df.plot(x='x',y='y',kind='scatter', title='Test Cluster')

In [ ]:
cluster_repr_old = butils.delta_medoids_one_shot(df, 0.2, distance.cosine)
cluster_repr = butils.delta_medoids_full(df, 0.02, distance.cosine)

In [ ]:
cluster_repr

In [ ]:
#cluster_repr.plot(x='x',y='y',kind='scatter', title='cluster_repr')
#cluster_repr_old.plot(x='x',y='y',kind='scatter', title='cluster_repr_old')